In [1]:
import numpy as np
import numpy as np
import pandas as pd
import inspect
import os
np.random.seed(0)

#Activation functions

def sin_function(x):
    return np.sin(x)

def relu(x):
    return np.maximum(0, x)

def tanh(x):
    return np.tanh(x)

#Function to approximate
def f(x):
    return np.sin(3*np.pi*x + 3*np.pi/20) * np.cos(2*np.pi*x + np.pi/10) + 2


In [2]:
#Function to build the mass matrix - M
from scipy.integrate import quad

def mass_matrix(func,sigma, weights, biases, n):
    M = np.zeros([n, n])
    
    for i in range(0, n):
        for j in range(0, n):
            M[i, j] = quad(func, 0, 1, args=(weights, biases,i+1,j+1,sigma), limit=1000)[0]

    return M

In [3]:
#Single basis function - Neural Network
def single_neural_net(x,weights,biases,i, sigma): 

    return sigma(weights[i]*x+biases[i])


In [4]:
#Basis function multiplication helper for quadrature
def double_neural_net(x,weights,biases,i,j, sigma): 

    return single_neural_net(x,weights,biases,i,sigma)*single_neural_net(x,weights,biases,j,sigma)

In [5]:
#Bias helper function for quadrature
def b_neural_net(x, weights, biases,i, sigma,f):
    return single_neural_net(x,weights,biases,i, sigma)*f(x)


In [6]:
#quadrature for the bias matrix - B
def b_matrix(func,sigma, weights, biases,f, neurons):
    b = np.zeros([neurons, 1])
    for i in range(0, neurons):
            b[i] = quad(func, 0, 1, args=(weights, biases,i,sigma,f), limit=1000)[0]

    return b

In [7]:
#Function to build the approximation vector
def u_nn(x, weights, biases, sigma,collocation, neurons, a):
    u = np.zeros(collocation)
    
    for j in range(0,collocation):
        for i in range(0,neurons):
            
            u[j] += single_neural_net(x[j], weights, biases, i, sigma)*a[i]
        
    return u


In [8]:
#helper function for eigenvalues and eigenvectors
from scipy.linalg import eig
def get_eig(M):
    D,V = eig(M, left=False, right=True)


    idx = D.argsort()[::-1]   
    D = D[idx]
    V = V[:,idx]

    D = np.flip(D)
    V = np.fliplr(V)

    return D,V

In [9]:
#Function to calculate L2 loss
def l2_loss(f_x, u_nn_result):
    return np.linalg.norm(f_x - u_nn_result)**2

In [10]:
#Function to build the Phi matrix, made up of n basis functions
def phi_matrix(x,weights,biases,sigma,neurons,collocation):
    phi = np.zeros((collocation,neurons))
    for i in range(0,neurons):
        for j in range(0,collocation):

            phi[j,i] = sigma(weights[i]*x[j]+biases[i])
    return phi


In [11]:
def evaluate_func_approx(x,f, weights, biases, sigma, neurons, collocation,verbose=False):
    phi = phi_matrix(x,weights,biases,sigma,neurons,collocation)
    f_func = f(x)
    a = np.linalg.pinv(phi)@f_func

    u = u_nn(x, weights, biases, sigma, collocation,neurons, a)

    loss = l2_loss(f_func, u)
    if verbose:
        print("="*20)
        print("Sigma:", sigma.__name__)
        print("Number of collocation points:", collocation)
        print("Number of neurons:", neurons)
        print("L2 Loss:", loss)
        print("="*20)
    return loss

In [12]:
def export_list(data, list_name, sheet_name, directory='/mnt/c/Git_Repos/Function_approximation/Results', columns=["Loss", "# of Collocation", "# of Neurons", "Activation Function", "R - Sample Range"]):
    file_name = f"{list_name}.xlsx"
    full_path = os.path.join(directory, file_name) if directory else file_name
    df = pd.DataFrame(data, columns=columns)

    if os.path.exists(full_path):
        try:
            # If the file exists, load the workbook and append the data to a new sheet
            with pd.ExcelWriter(full_path, engine='openpyxl', mode='a') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        except ValueError as e:
            # If the sheet already exists, print a message and continue
            print(f"Sheet name '{sheet_name}' already exists in the file '{file_name}'")
    else:
        # If the file does not exist, create a new one
        df.to_excel(full_path, index=False, sheet_name=sheet_name)

    print(f"Data exported to {full_path}")

In [13]:
import matplotlib.pyplot as plt

def compute_loss_values(neurons, collocations, sigmas, Rs,sheet_name,xmin=0,xmax=1,verbose=False):
    loss_values = []
    #np.random.seed(0)  # Set a seed for reproducibility
    for sigma in sigmas:
        for neuron in neurons:
            for collocation in collocations:
                for R in Rs:
                    x = np.linspace(xmin, xmax, collocation)
                    weights = np.random.uniform(-R, R, neuron)
                    biases = np.random.uniform(-R, R, neuron)
                    loss = evaluate_func_approx(x, f, weights, biases, sigma, neuron, collocation, verbose=verbose)
                    loss_values.append((loss, collocation, neuron, sigma.__name__,R))  # Add activation function name
    export_list(loss_values, "Func_Approx_Method_Results",sheet_name)
    return loss_values

In [14]:
def print_loss_by_collocation(loss_values):
    """
    Prints the loss values grouped by collocation point, activation function, and range R.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each collocation point
    collocation_loss_data = {collocation: [] for _, collocation, _, _, _ in loss_values}

    # Populate the dictionary with (neuron, loss, activation_func_name, R) tuples for each collocation point
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        collocation_loss_data[collocation].append((neuron, loss, activation_func_name, R))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the collocation points and print the relevant data
        for collocation, data in collocation_loss_data.items():
            print(f"# of Collocation points: {collocation}")
            
            # Print the neurons, loss values, and R for the current activation function
            for neuron, loss, act_func_name, R in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Neurons: {neuron}, Loss: {loss:.10e}, R: {R}")



In [15]:
def print_loss_by_neuron(loss_values):
    """
    Prints the loss values grouped by neuron, activation function, and range R.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each neuron
    neuron_loss_data = {neuron: [] for _, _, neuron, _, _ in loss_values}

    # Populate the dictionary with (collocation, loss, activation_func_name, R) tuples for each neuron
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        neuron_loss_data[neuron].append((collocation, loss, activation_func_name, R))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the neurons and print the relevant data
        for neuron, data in neuron_loss_data.items():
            print(f"# of Neurons: {neuron}")
            
            # Print the collocation points, loss values, and R for the current activation function
            for collocation, loss, act_func_name, R in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Collocation points: {collocation}, Loss: {loss:.10e}, R: {R}")




In [16]:
def print_loss_by_R(loss_values):
    """
    Prints the loss values grouped by the range R, activation function, collocation points, and neurons.
    
    Parameters:
    loss_values (list of tuples): Each tuple contains (loss, collocation, neuron, activation_func_name, R).
    """
    # Initialize the dictionary to store data for each R value
    R_loss_data = {R: [] for _, _, _, _, R in loss_values}

    # Populate the dictionary with (collocation, neuron, loss, activation_func_name) tuples for each R value
    for loss, collocation, neuron, activation_func_name, R in loss_values:
        R_loss_data[R].append((collocation, neuron, loss, activation_func_name))

    # Iterate over the unique activation function names
    for activation_func_name in set(activation_func_name for _, _, _, activation_func_name, _ in loss_values):
        print(f"Activation Function: {activation_func_name}")
        
        # Iterate over the R values and print the relevant data
        for R, data in R_loss_data.items():
            print(f"Range R: {R}")
            
            # Print the collocation points, neurons, and loss values for the current activation function
            for collocation, neuron, loss, act_func_name in data:
                if act_func_name == activation_func_name:
                    print(f"  # of Collocation points: {collocation}, # of Neurons: {neuron}, Loss: {loss:.10e}")

In [17]:
collocation = 100
neurons = 50
sigma = sin_function
x = np.linspace(0,1,collocation)
weights = np.random.uniform(-1,1,neurons)
biases = np.random.uniform(-1,1,neurons)
test = evaluate_func_approx(x,f, weights, biases, sigma, neurons, collocation,verbose=True)

Sigma: sin_function
Number of collocation points: 100
Number of neurons: 50
L2 Loss: 0.08512083411934636


# Test 1 - Increasing #of neurons
Keeping collocations fixed at 50, and using ReLU, tanh, and sin as activation functions

In [18]:
neurons = [5,10,20,30,40,50,100]
sigmas = [relu,tanh,sin_function]
collocations = [50]
Rs = [1]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,"Increasing Neurons",verbose=False)
#print_loss_by_R(loss_values)
#loss_values

Sheet name 'Increasing Neurons' already exists in the file 'Func_Approx_Method_Results.xlsx'
Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Func_Approx_Method_Results.xlsx


# Test 2 - Increasing Collocation points
Keeping neurons fixed at 20, and the same activation functions

In [19]:
neurons = [20]
sigmas = [tanh,relu,sin_function]
collocations = [10,50,100,200,500]
Rs = [1]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,"Increasing Collocation",verbose=False)

Sheet name 'Increasing Collocation' already exists in the file 'Func_Approx_Method_Results.xlsx'
Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Func_Approx_Method_Results.xlsx


In [20]:
#print_loss_by_neuron(loss_values)

# Test 3 - Increasing the sampling range R
Keeping neurons and collocations fixed, increasing R to increase chance that w_i and b_i are further away from w_j and b_j

In [21]:
neurons = [30]
sigmas = [tanh,relu,sin_function]
collocations = [50]
Rs = [1,2,4,5,10]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,"Increasing Sampling Range",verbose=False)

Sheet name 'Increasing Sampling Range' already exists in the file 'Func_Approx_Method_Results.xlsx'
Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Func_Approx_Method_Results.xlsx


In [22]:
#print_loss_by_collocation(loss_values)

In [23]:
neurons = [10,20,30,50,100]
sigmas = [tanh,relu,sin_function]
collocations = [5,10,30,50,100]
Rs = [1,2,4,5,10]
loss_values = compute_loss_values(neurons, collocations,sigmas,Rs,"Full_Test",verbose=False)

Sheet name 'Full_Test' already exists in the file 'Func_Approx_Method_Results.xlsx'
Data exported to /mnt/c/Git_Repos/Function_approximation/Results/Func_Approx_Method_Results.xlsx


In [24]:
#print_loss_by_collocation(loss_values)

# Building Mass matrix

In [25]:
from scipy.integrate import quad
from numpy.linalg import cond
from scipy.linalg import eig

def mass_matrix(func,sigma, weights, biases, neurons):
    M = np.zeros([neurons, neurons])
    
    for i in range(0, neurons):
        for j in range(0, neurons):
            M[i, j] = quad(func, 0, 1, args=(weights, biases,i,j,sigma), limit=1000)[0]

    return M

In [26]:
neurons = 20
R=10
collocation = 10
x = np.linspace(0,1,collocation)
weights = np.random.uniform(-R,R,neurons)
biases = np.random.uniform(-R,R,neurons)
sigma = sin_function
M = mass_matrix(double_neural_net,sigma, weights, biases, neurons)
M

array([[ 0.41674024, -0.11257135, -0.31010403,  0.35565366,  0.4666178 ,
        -0.06564373,  0.03690916,  0.11609082,  0.05956116,  0.08696524,
        -0.05841472, -0.03942721,  0.03068472,  0.41779842,  0.42155246,
         0.41778513,  0.49071799,  0.01929366, -0.0371087 ,  0.12310746],
       [-0.11257135,  0.54251744, -0.08207161, -0.03477159,  0.01379782,
        -0.09330023,  0.24590998, -0.33755141, -0.15523795, -0.45037547,
         0.50608706,  0.26608715, -0.224019  , -0.08302562, -0.08441021,
         0.06410157, -0.12016964, -0.02202749,  0.44658847,  0.14873242],
       [-0.31010403, -0.08207161,  0.51284027, -0.4786496 , -0.46479543,
         0.39717221,  0.04032458, -0.02229856,  0.33350763,  0.1170935 ,
        -0.19978534, -0.0905107 ,  0.09716657, -0.46171405, -0.20921444,
        -0.40793953, -0.33597419,  0.10967025, -0.26968949, -0.45816767],
       [ 0.35565366, -0.03477159, -0.4786496 ,  0.47943313,  0.47123124,
        -0.34157639, -0.08520479,  0.09443258, -

In [27]:
det_m = np.linalg.det(M)
det_m

-7.473292981429498e-185

In [28]:
cond_m = np.linalg.cond(M)
cond_m

1.0057548795552702e+19

In [29]:
b = b_matrix(b_neural_net,sigma,weights, biases,f,neurons)
#b

In [30]:
a = np.linalg.solve(M,b)
#a

In [31]:
u = u_nn(x,weights,biases,sigma,collocation,neurons,a)

/tmp/ipykernel_23215/1981966750.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  u[j] += single_neural_net(x[j], weights, biases, i, sigma)*a[i]


In [32]:
u

array([2.43918305, 2.53021268, 1.92421898, 2.33745398, 2.99751064,
       2.43020355, 1.90419358, 2.47027579, 2.50522261, 1.57880669])

In [33]:
loss = l2_loss(f(x), u)
print("L2 Loss:", loss)

L2 Loss: 0.000173335145905158
